In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time 
from transformers import pipeline
import en_core_web_sm

!pip install https://huggingface.co/spacy/en_core_web_sm/resolve/main/en_core_web_sm-any-py3-none-any.whl

     --------------------------------------- 12.8/12.8 MB 20.4 MB/s eta 0:00:00


In [15]:
LINK2 = "https://easychair.org/smart-program/IC2S2-2021/talk_author_index.html"
r = requests.get(LINK2)
soup2021 = BeautifulSoup(r.content)
participants2021 = []
for i in range(2000):
    participants2021.append(soup2021.find_all(href = "person"+str(i)+".html"))
participants2021_text = []
for name in participants2021:
    if len(name) == 0:
        pass
    else:
        participants2021_text.append(name[0].get_text())

print(len(participants2021_text))

716


In [16]:
LINK = "https://2019.ic2s2.org/delegates/"
r = requests.get(LINK)
soup2019 = BeautifulSoup(r.content)
participants2019 = soup2019.find_all('td')
participants2019 = participants2019[2:]
participants2019 = participants2019[::2]
participants2019_text = []
for name in participants2019:
    participants2019_text.append(name.get_text())
    
participants2019_DELEGATES_text = participants2019_text.copy()








Link2019Posters = "https://2019.ic2s2.org/posters/"
LINK = Link2019Posters
r = requests.get(LINK)
soup2019 = BeautifulSoup(r.content)
soupLI = soup2019.find_all('li')
participants_POSTERLINK_text_2019 = []
for i in range(32, len(soupLI)):
    try:
        helper = soupLI[i].next_element
        And_names = helper.split("and")

        if len(And_names) > 1:
            comma_names = And_names[0].split(",")

            comma_names.append(And_names[1])
            #comma_names.extend(And_names[1])

            total_names = comma_names.copy()
        else:
            total_names = And_names.copy()
        
        
        for name in total_names:
            if name[0] == " ":
                name = name[1:]
            if name[-1] == " ":
                name = name[:-1]

            participants_POSTERLINK_text_2019.append(name)
    except: 
        pass 

    
    
    
    
nlp = en_core_web_sm.load()
def intelligent_name_collection(text):
    doc = nlp(text)

    names_of_presenters = []
    name = ""
    doing_a_name = False
    ready_for_new_name = True
    for token in doc:    

        if token.ent_type_ == "PERSON":
            if ready_for_new_name:
                name = token.text
                ready_for_new_name = False
                doing_a_name = True
            else:
                name = name + " " + token.text

        if token.ent_type_ != "PERSON":

            if doing_a_name:
                names_of_presenters.append(name)

                doing_a_name = False
                ready_for_new_name = True
    
    return names_of_presenters


Link2019Presentations = "https://2019.ic2s2.org/oral-presentations/"
LINK = Link2019Presentations
r = requests.get(LINK)
soup2019 = BeautifulSoup(r.content)
soup = soup2019.find_all('p')

participants_PRESENTATIONS_text_2019 = []

for e in range(3, len(soup)):
    
    split_text = soup[e].get_text()
    
    names_of_presenters_gathered = intelligent_name_collection(split_text)
    
    participants_PRESENTATIONS_text_2019.extend(names_of_presenters_gathered)
    


participants2019_text = []
participants2019_text.extend(participants2019_DELEGATES_text)
participants2019_text.extend(participants_POSTERLINK_text_2019)
participants2019_text.extend(participants_PRESENTATIONS_text_2019)

print(len(participants2019_text))

print(participants_PRESENTATIONS_text_2019[0:10])
print(participants_POSTERLINK_text_2019[0:10])

1598
['Ziv Epstein11:00 –', 'Ding', 'Jingwen Zhang', 'Vaccine Misinformation', 'Juhi Kulshrestha', 'Abhijnan Chakraborty', 'Meeyoung Cha', 'Krishna Gummadi', 'Analyzing Biases', 'Alexandre Bovet']
['Frank Peter Pijpers', 'Martin Lukac', 'André Grow', 'Kiran Garimella', 'Gianmarco De Francisci Morales', 'Aristides Gionis', 'Michael Mathioudakis', 'Eaman Jahani', 'Peter Krafft', 'Yoshihiko Suhara']


In [19]:
LINK3 = "https://docs.google.com/spreadsheets/u/0/d/e/2PACX-1vTX9_1Xftn7D-nSI8X9b7tafr_Z0kAbphKdfZ8qUSU9p-syXNsGPdhHl5ZyTnKKL-T6dCEJqtsrn3wy/pubhtml/sheet?headers=false&gid=181378784"

tables = pd.read_html(LINK3, header=1)
Presenters = tables[0]["Presenters"]

participants_2020 = []
for i in range(len(Presenters)):
    if type(Presenters[i]) == str:
        for name in Presenters[i].split(","):
           participants_2020.append(name)

print(len(participants_2020))

1389


In [20]:

participantsALL_text = []
participantsALL_text.extend(participants2019_text.copy())
participantsALL_text.extend(participants_2020.copy())
participantsALL_text.extend(participants2021_text.copy())

pandasNames = pd.DataFrame(participantsALL_text, columns=["Names"])


print(len(pandasNames["Names"]))
pandasNames.drop_duplicates()
pandasNames["Names"]


3703


0       Abbasiharofteh, Milad
1                 Adam, Silke
2           Aggarwal, Anupama
3          Aiello, Luca Maria
4       Akbaritabar, Aliakbar
                ...          
3698          Kreuter, Frauke
3699          Mai, Tung-Duong
3700            Neffke, Frank
3701        Secchini, Valeria
3702          Thurner, Stefan
Name: Names, Length: 3703, dtype: object

Day 2

In [ ]:

BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION = "v1/"
RESOURCE = "author/search"   #author or paper

authorIDs_list = []
authorColabIDs_list = []
i = 0
loop_fails = 0
author_count = 0
author_colab_count = 0
for name in pandasNames["Names"]:
    print(i)
    
    params = {'query':name,     #name, because we search by author
               "offset":0,               #offset dunno
               "limit":1,                   #limit is not neccessary but gives the number
                "fields":"papers.authors"}      #we can search for any of the fields given at semantic scholar https://api.semanticscholar.org/api-docs/graph#tag/Paper-Data/operation/get_graph_get_paper_references
    
    try:
        
        my_url = BASE_URL + VERSION + RESOURCE
        r = requests.get(my_url, params=params)
        json_file = r.json()["data"]
        
        authorId = json_file[0]["authorId"]
        author_count += 1
        
        
        for paper_i in range(len(json_file[0]["papers"])):
            for auth in range(len(json_file[0]["papers"][paper_i]["authors"])):
                try:
                    colab_id = json_file[0]["papers"][paper_i]["authors"][auth]["authorId"]
                    authorColabIDs_list.append(colab_id)
                    author_colab_count += 1
                except:
                    pass
        authorIDs_list.append(authorId)

        print(author_colab_count)
    except:
        print("failed: ", i)
        loop_fails += 1
    
    time.sleep(5*60 / 100 + 0.1)
    
    i+=1
    
    
with open(r'C:\Users\Bbjar\OneDrive\Skrivebord\Semester6\Comp Social Science\AuthorID.txt', 'w') as fp:
    for item in authorIDs_list:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'C:\Users\Bbjar\OneDrive\Skrivebord\Semester6\Comp Social Science\AuthorColabsID.txt', 'w') as fp:
    for item in authorColabIDs_list:
        # write each item on a new line
        fp.write("%s\n" % item)

        

    #100 per 5 minutes

0
26
1
612
2
616
3
1187
4
1217
5
1223
6
1263
7
1303
8
1402
9
1410
10
1424
11
1445
12
1627
13
failed:  13
14
1638
15
1649
16
1652
17
1675
18
1848
19
1874
20
2233
21
2439
22
2445
23
failed:  23
24
2594
25
3113
26
3137
27
failed:  27
28
3186
29
3192
30
failed:  30
31
3201
32
failed:  32
33
3359
34
3459
35
3471
36
failed:  36
37
4029
38
4035
39
4056
40
4058
41
4086
42
4190
43
4198
44
4207
45
4230
46
4240
47
failed:  47
48
4320
49
4384
50
4427
51
4594
52
failed:  52
53
4776
54
4785
55
5111
56
7036
57
8066
58
8320
59
8343
60
8407
61
8413
62
8476
63
failed:  63
64
8596
65
8927
66
8955
67
failed:  67
68
8968
69
9035
70
9103
71
9234
72
failed:  72
73
9522
74
9525
75
9851
76
9941


Consider the list of author ids you have found in the exercise above. For each author, use the Academic Graph API to find:

their aliases
their name
their papers, where for each paper we want to retain:
title
abstract
the year of publication
the externalIds (this is because there are universal identifiers for scientific works called DOI that we can use across platforms)
s2FieldsOfStudy the fields of study
citationCount the number of times that this paper was cited
(Hint: you can find authors in batches)

In [82]:

file1 = open(r'C:\Users\Bbjar\OneDrive\Skrivebord\Semester6\Comp Social Science\AuthorID.txt', 'r')
Lines1 = file1.readlines()
authorID = []

file2 = open(r'C:\Users\Bbjar\OneDrive\Skrivebord\Semester6\Comp Social Science\AuthorColabsID.txt')
Lines2 = file2.readlines()
authorColabsID = []

for line in Lines1:
    authorID.append(line[:-1])
for line in Lines2:
    authorColabsID.append(line[:-1])


all_authers_list = authorColabsID.copy()
all_authers_list.extend(authorID)
all_authers_list = list(set(all_authers_list))
print(len(all_authers_list))


91254


From adrian 

In [353]:

def author_df(data):
    helperdict1 = {'authorId':[],'name':[], 'aliases':[], 'citationCount':[], 'field':[]}

    for person_index in range(len(data)):
        authorId = data[person_index]['authorId']
        name  = data[person_index]['name']
        aliases  =  data[person_index]['aliases']
        helperFields_momentary = []
        citation_count = 0
        for paper_index in range(len(data[person_index]['papers'])):
            try:
                s2FieldsOfStudy = data[person_index]['papers'][paper_index]['s2FieldsOfStudy'][0]["category"]
                helperFields_momentary.append(s2FieldsOfStudy)
            except:
                pass
            
            citation_count += data[person_index]['papers'][paper_index]['citationCount']
        try:
            the_mode = mode(helperFields_momentary)
        except:
            the_mode = None
        helperdict1['field'].append(the_mode)
        helperdict1['citationCount'].append(citation_count)
        helperdict1['authorId'].append(authorId)
        helperdict1['name'].append(name)
        helperdict1['aliases'].append(aliases)
        
    return pd.DataFrame(helperdict1)

def paper_df(data):
    helperdict2 = {'paperId':[],'title':[], 'year':[], 'externalId':[], 'citationCount':[], 'fields':[], "authorIds":[]}

    for person_index in range(len(data)):    
        for paper_index in range(len(data[person_index]['papers'])):
            try:
                paperId = data[person_index]['papers'][paper_index]['paperId']
                title = data[person_index]['papers'][paper_index]['title']
                year  = data[person_index]['papers'][paper_index]['year']
                doi = data[person_index]['papers'][paper_index]['externalIds']['DOI']
                citationCount = data[person_index]['papers'][paper_index]['citationCount']
                
                helper1 = []
                for i in range(len(data[person_index]['papers'][paper_index]['s2FieldsOfStudy'])):
                    helper1.append(data[person_index]['papers'][paper_index]['s2FieldsOfStudy'][i]["category"])

                helper2 = []
                for i in range(len(data[person_index]['papers'][paper_index]['authors'])):
                    helper2.append(data[person_index]['papers'][paper_index]['authors'][i]['authorId'])
                
                
                def fail_if_other_dataframe_would_fail():
                    #this cuts of around 15% of the data as we see through testing

                    #This is to make the statement fail if abstract dataframe would also fail
                    data[person_index]['papers'][paper_index]['abstract']
                fail_if_other_dataframe_would_fail()
                
                #appending
                helperdict2['paperId'].append(paperId)
                helperdict2['title'].append(title)
                helperdict2['year'].append(year)
                helperdict2['externalId'].append(doi)
                helperdict2['citationCount'].append(citationCount)
                helperdict2["fields"].append(helper1)
                helperdict2["authorIds"].append(helper2)
            except:
                pass 

    return pd.DataFrame(helperdict2)

def abstract_df(data):
    helperdict3 = {'paperId':[],'abstract':[]}
    g = 0
    for person_index in range(len(data)):
        
        for paper_index in range(len(data[person_index]['papers'])):
            
            g += 1
            try:
                
                def fail_if_other_dataframe_would_fail():
                    #this is to make the statement fail if it would fail for paper dataframe
                    #this cuts of around 15% of the data as we see through testing
                    paperId = data[person_index]['papers'][paper_index]['paperId']
                    title = data[person_index]['papers'][paper_index]['title']
                    year  = data[person_index]['papers'][paper_index]['year']
                    doi = data[person_index]['papers'][paper_index]['externalIds']['DOI']
                    citationCount = data[person_index]['papers'][paper_index]['citationCount']

                    helper1 = []
                    for i in range(len(data[person_index]['papers'][paper_index]['s2FieldsOfStudy'])):
                        helper1.append(data[person_index]['papers'][paper_index]['s2FieldsOfStudy'][i]["category"])

                    helper2 = []
                    for i in range(len(data[person_index]['papers'][paper_index]['authors'])):
                        helper2.append(data[person_index]['papers'][paper_index]['authors'][i]['authorId'])
                fail_if_other_dataframe_would_fail()


                
                
                
                
                
                #actual stuff we need to do in this dataframe
                
                paperId  =  data[person_index]['papers'][paper_index]['paperId']
                abstract  =  data[person_index]['papers'][paper_index]['abstract']
                helperdict3['paperId'].append(paperId)
                helperdict3['abstract'].append(abstract)
            except:
                pass
    print(g)
    return pd.DataFrame(helperdict3)

The stuff below is for testing!

In [358]:
my_url = BASE_URL + VERSION + RESOURCE
BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION = "v1/"
RESOURCE = "author/batch?"   #author or paper

params = "fields=aliases,name,papers.title,papers.abstract,papers.year,papers.externalIds,papers.s2FieldsOfStudy,papers.citationCount,papers.authors"
my_url = BASE_URL + VERSION + RESOURCE
for i in range(1):
    json_data = {'ids':all_authers_list[i*10:i*10 + 10]}
    r = requests.post(my_url, json = json_data, params = params)
    data = r.json()



#authorDataframe = author_df(data)
#print(authorDataframe)


In [354]:
BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION = "v1/"
RESOURCE = "author/batch?"   #author or paper
params = "fields=aliases,name,papers.title,papers.abstract,papers.year,papers.externalIds,papers.s2FieldsOfStudy,papers.citationCount,papers.authors"
k = True

for i in range(int(len(all_authers_list)/10)):
    my_url = BASE_URL + VERSION + RESOURCE
    json_data = {'ids':all_authers_list[i*10:(i*10 )+10]}
    r = requests.post(my_url, json = json_data, params = params)
    data = r.json()
    
    
    if len(data) <= 1:
        pass
    else:
        if k:
            authorDataframe = author_df(data)
            paperDataframe = paper_df(data) 
            abstractDataframe = abstract_df(data)
            k = False
        else: 

            new_authorDataframe = author_df(data)
            authorDataframe = pd.concat([authorDataframe, new_authorDataframe], ignore_index=True, sort=False)

            new_paperDataframe = paper_df(data)
            paperDataframe = pd.concat([paperDataframe, new_paperDataframe], ignore_index=True, sort=False)


            new_abstractDatafram = abstract_df(data)
            abstractDataframe = pd.concat([abstractDataframe, new_abstractDatafram], ignore_index=True, sort=False)



            #authorDataframe.concat(author_df(data))
            #paperDataframe.concat(paper_df(data))
            #abstractDataframe.concat(abstract_df(data))

    time.sleep(5*60 / 100 + 0.1)

    if i >= 9:
        break


print(paperDataframe)  
print(abstractDataframe)


1135
918
369
527
448
485
832
645
948
842
                                       paperId  \
0     6685eb576941d36369c1dda4940a0e1de6d51582   
1     b7b8569eb8e841f0170fd690089e81a658c6c6b3   
2     1384fb54b8a5c5848e17da1a29a9f710122d222c   
3     347e89236ab2f1f1e0fa8b3967f8250c80d8f592   
4     a4cdb9cd7b56681784bc101097121c5aee7c6efa   
...                                        ...   
5889  af05425d0fd3f4fb4c07fa320baa411ebc3fdab1   
5890  42bb017d61d7057aeffbcb2771c1ca32e538ab41   
5891  39103838fda37dc0145301c9bb478f071e04f65e   
5892  63815c0655785919b998c795e7e3d072789591bc   
5893  8fa666dfcbb1c849020248716897cf992b7f633e   

                                                  title    year  \
0        DIALYSIS. PATHOPHYSIOLOGY AND CLINICAL STUDIES  2014.0   
1            Acute renal failure - clinical studies - 2  2009.0   
2           Collaborating with care in virtual sessions  2020.0   
3                                     Contemporary IMRT  2019.0   
4     Developments in I

In [355]:
print(authorDataframe)


      authorId             name  \
0   2080725882      C. A. Beati   
1   2073301487          L. Toma   
2   2053433279     Sonia Graham   
3    145071876          S. Webb   
4      2088358   Mario Paolucci   
..         ...              ...   
95    21778810       K. Paarnio   
96  2111477347     David Sionit   
97     1923302  R. Angus Silver   
98   100716450    Jeff Huizinga   
99     1720970       V. Catania   

                                              aliases  citationCount field  
0                                                None              0  None  
1                  [L. Tomá, Luca Di Toma, Luca Toma]              1  None  
2                                                None              0  None  
3   [S Webb, S. Webb, Steve Webb, Stephen E. D. Webb]          11868  None  
4                                       [M. Paolucci]           2038  None  
..                                                ...            ...   ...  
95         [K Paarnio, K. Paarnio, Karol

In [357]:
authorDataframe.to_csv('authorDF.csv', index=False)
paperDataframe.to_csv('paperDF.csv', index=False)
abstractDataframe.to_csv('abstractDF.csv', index=False)


Det vi ved er at vi kan søge efter op til 1000 papers at a time 

resource = "author/batch"

spørgsmål.
Hvordan skal params se ud ifølge deres API https://api.semanticscholar.org/api-docs/graph#tag/Author-Data/operation/get_graph_get_author_search

Husk at fjerne \n for linjerne via [:-1]




7 mil papers
180 thousand authors
